In [215]:
%load_ext autoreload
%autoreload 2
PROJ_HOME='/Users/keemsunguk/Projects/analyst_performance/'
import sys
sys.path.append(PROJ_HOME)
import gc
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
import locale
import pandas as pd
import copy
import pickle
from tqdm import tqdm

In [217]:
from pathlib import Path

In [218]:
data_dir = Path(PROJ_HOME+'/data')

In [219]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [220]:
nlp = spacy.load("en_core_web_lg")

In [221]:
from googletrans import Translator

In [222]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])

# Loading sentiment data

In [223]:
sentiment_df = pd.read_pickle(PROJ_HOME+'/data/sentiment_pub_trade.pkl')

In [224]:
sentiment_df['author'] = sentiment_df['author'].apply(lambda x: x.strip())

In [225]:
author_production = dict(sentiment_df['author'].value_counts())

In [226]:
author_production

{'이동헌': 354,
 '박종선': 346,
 '한상원': 280,
 '김경민': 271,
 '조미진': 268,
 '박상준': 258,
 '한유정': 257,
 '이상헌': 255,
 '김현용': 255,
 '정태준': 249,
 '박은정': 243,
 '안재민': 241,
 '정홍식': 237,
 '이동욱': 227,
 '김홍식': 226,
 '양지환': 215,
 '김회재': 214,
 '백영찬': 210,
 '손효주': 208,
 '최종경': 206,
 '성종화': 199,
 '김운호': 197,
 '홍세종': 197,
 '박세라': 196,
 '유정현': 196,
 '안지영': 193,
 '박신애': 192,
 '이경신': 192,
 '박강호': 192,
 '심은주': 192,
 '하누리': 191,
 '최민하': 188,
 '김동원': 186,
 '박종렬': 186,
 '강승건': 183,
 '강동진': 183,
 '신수연': 182,
 '박성호': 182,
 '서정연': 179,
 '김세련': 177,
 '백재승': 173,
 '오동환': 173,
 '유지웅': 172,
 '이문종': 170,
 '윤재성': 169,
 '김고은': 168,
 '남성현': 167,
 '김장열': 167,
 '박형우': 166,
 '김지산': 162,
 '이상현': 161,
 '강성진': 160,
 '주영훈': 160,
 '이순학': 160,
 '박혜진': 160,
 '김미송': 158,
 '김지영': 158,
 '이진협': 158,
 '전우제': 157,
 '박은경': 154,
 '황어연': 152,
 '김현태': 152,
 '임은영': 152,
 '김혜미': 151,
 '송유림': 150,
 '이한준': 149,
 '유재선': 149,
 '이기훈': 149,
 '배송이': 148,
 '김동하': 148,
 '이종형': 147,
 '남효지': 147,
 '정동익': 146,
 '김장원': 146,
 '한병화': 145,
 '어규진': 144,
 '장문수': 144,

In [227]:
sentiment_df['recommendation'].value_counts()

BUY              23211
매수                7057
NR                6258
HOLD              1556
중립                 388
MarketPerform      263
Trading Buy        158
Neutral             99
Outperform          96
STRONG BUY          67
없음                  59
보유                  27
Underperform        11
비중축소                 8
Sell                 5
REDUCE               4
Name: recommendation, dtype: int64

In [228]:
recommendation_lookup = {
    'STRONG BUY': 'STRONG_BUY',
    'Strong Buy': 'STRONG_BUY',
    'Trading Buy': 'BUY',
    'BUY': 'BUY',
    '매수': 'BUY',
    'Outperform': 'BUY',
    '보유': 'HOLD',
    'NEUTRAL': 'HOLD',
    'HOLD': 'HOLD',
    '중립': 'HOLD',
    'MarketPerform': 'HOLD',
    'Neutral': 'HOLD',
    '없음': 'NR',
    'NR': 'NR',
    '비중축소': 'REDUCE',
    'REDUCE': 'REDUCE',
    'Underperform': 'SELL',
    'Sell': 'SELL',
}

In [229]:
sentiment_df['recommendation'] = sentiment_df['recommendation'].apply(lambda x: recommendation_lookup[x])

In [230]:
sentiment_df[['recommendation']].value_counts()

recommendation
BUY               30522
NR                 6317
HOLD               2333
STRONG_BUY           67
SELL                 16
REDUCE               12
dtype: int64

In [231]:
sentiment_df

,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,closing,...,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title,sentiment
0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 실적은 기대수준을 소폭 하회할 전망,51300,50000,...,41000.0,39400.0,40450.0,40450.0,40550.0,36400.0,51400,51400,4Q09 earnings are expected to meet expectations,-0.100000
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,폴리실리콘 원가개선 필요한 시점,62700,62800,...,52000.0,50100.0,48450.0,48200.0,46300.0,39150.0,65900,67300,Polysilicon cost improvement required,0.000000
2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,"견조한 고무사업, 그러나 부진한 페놀유도체",77300,76000,...,65900.0,64200.0,62800.0,63200.0,63700.0,56300.0,82100,82100,"A solid rubber business, but sluggish phenol oil",0.000000
3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,20년 연초에 꾸는 꿈,237000,232000,...,323500.0,305500.0,298000.0,301500.0,291500.0,284500.0,322500,351000,Dreams of 20 years,0.000000
4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 ESS 비용반영을 긍정적으로 보자,321000,314000,...,404000.0,385500.0,380000.0,377500.0,367000.0,343500.0,391500,422500,Let's see a positive reflection of ESS costs i...,0.227273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,182500,188000,...,0.0,0.0,0.0,0.0,0.0,0.0,"188,000","188,000",Room and comfortable 2022,0.400000
39796,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,10850,10950,...,0.0,0.0,0.0,0.0,0.0,0.0,"10,950","10,950",Welcome to the confusion of the confidence,0.800000
39797,심은주,하나금융,A271560,오리온,9/10/21,9/9/21,9/8/21,저가 매수 유효,124500,123500,...,0.0,0.0,0.0,0.0,0.0,0.0,"124,500","124,500",Low cost volume valid,0.000000
39798,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지",8840,8770,...,0.0,0.0,0.0,0.0,0.0,0.0,"8,880","8,880","Hidden pearls, 3Q Human & Renewable Energy",-0.083333


In [232]:
sentiment_df.columns.to_list()

['author',
 'source',
 'ticker_symbol',
 'company',
 'report_date',
 '발행 전일',
 '발행 전전일',
 'report_title',
 'opening',
 'closing',
 '고가',
 '전일종가',
 '전전일종가',
 'recommendation',
 '고가1',
 '고가2',
 '고가3',
 '고가4',
 '고가5',
 '고가6',
 '고가7',
 '고가8',
 '고가9',
 '고가10',
 '고가11',
 '고가12',
 '고가13',
 '고가14',
 '고가15',
 '고가16',
 '고가17',
 '고가18',
 '고가19',
 '고가20',
 '고가21',
 '고가22',
 '고가23',
 '고가24',
 '고가25',
 '고가26',
 '고가27',
 '고가28',
 '고가29',
 '고가30',
 '고가31',
 '고가32',
 '고가33',
 '고가34',
 '고가35',
 '고가36',
 '고가37',
 '고가38',
 '고가39',
 '고가40',
 '고가41',
 '고가42',
 '고가43',
 '고가44',
 '고가45',
 '고가46',
 '고가47',
 '고가48',
 '고가49',
 '고가50',
 'H25',
 'H50',
 'en_report_title',
 'sentiment']

In [233]:
new_col_name = {
    '발행 전일': 'report_date-1',
    '발행 전전일': 'report_date-2',
    '고가': 'high',
    '전일종가': 'closing-1',
    '전전일종가': 'closing-2',
}

In [234]:
sentiment_df.rename(columns =new_col_name, inplace=True)

In [235]:
new_col_name = {
    'closing-1': 'closing_1',
    'closing-2': 'closing_2',
}
sentiment_df.rename(columns =new_col_name, inplace=True)

In [236]:
# dont need this any more.  took care of them in excel
#locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
#locale.atoi(sentiment_df.at[39795, 'opening'].strip())

In [237]:
sentiment_df.at[23612, 'closing']

671000

In [241]:
sentiment_df.query('opening == 0')

,author,source,ticker_symbol,company,report_date,report_date-1,report_date-2,report_title,opening,closing,...,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title,sentiment
1592,이민희,IBK,A294630,서남,2020-01-29 00:00:00,2020-01-28 00:00:00,2020-01-23 00:00:00,구리선을 대체할 고온초전도 선재 선도기업,0,0,...,3140.0,3020.0,2845.0,3060.0,2985.0,3035.0,5200,5200,High temperature superconducting wire leading ...,0.16
2196,최종경,흥국,A294630,서남,2020-02-04 00:00:00,2020-02-03 00:00:00,2020-01-31 00:00:00,초전도 선재 상용화 선도기업,0,0,...,2985.0,3035.0,2965.0,3400.0,3345.0,3165.0,5200,5200,Superconducting leading companies,0.00
2197,나승두,SK,A294630,서남,2020-02-04 00:00:00,2020-02-03 00:00:00,2020-01-31 00:00:00,고온 초전도 시장의 성장 잠재력에 주목한다,0,0,...,2985.0,3035.0,2965.0,3400.0,3345.0,3165.0,5200,5200,Notice the growth potential of the high temper...,0.08
2675,최종경,흥국,A294140,레몬,2020-02-10 00:00:00,2020-02-07 00:00:00,2020-02-06 00:00:00,나노 멤브레인의 시대를 선도한다,0,0,...,17650.0,16400.0,14950.0,14650.0,14600.0,15000.0,13400,23200,Leading the era of nano membrane,0.00
2678,김장열,상상인,A294140,레몬,2020-02-10 00:00:00,2020-02-07 00:00:00,2020-02-06 00:00:00,김장열의 SEE-IT Note: 이 정도 성장 잠재력이라면 시총 ~5000억은 (공...,0,0,...,17650.0,16400.0,14950.0,14650.0,14600.0,15000.0,13400,23200,See-IT NOTE for Kim Jang Yu: If this is a pote...,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23078,정유진,IR큐더스,A347860,알체라,2020-12-10 00:00:00,2020-12-09 00:00:00,2020-12-08 00:00:00,밝은 인공지능 세상을 만들어가는 기업,0,0,...,23400.0,23050.0,22550.0,21900.0,21900.0,21750.0,50800,50800,Companies that make bright artificial intellig...,0.05
23079,정유진,IR큐더스,A347860,알체라,2020-12-10 00:00:00,2020-12-09 00:00:00,2020-12-08 00:00:00,공모분석보고서,0,0,...,23400.0,23050.0,22550.0,21900.0,21900.0,21750.0,50800,50800,Contact Analysis Report,0.00
23082,최종경,흥국,A357550,석경에이티,2020-12-10 00:00:00,2020-12-09 00:00:00,2020-12-08 00:00:00,고기능성 나노 소재 국가대표 강소기업,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,38000,38000,High Functional Nano Material National Represe...,0.16
23104,박종선,유진,A357550,석경에이티,2020-12-10 00:00:00,2020-12-09 00:00:00,2020-12-08 00:00:00,나노소재 핵심기술의 선도 업체,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,38000,38000,Leading of Nano Materials Core Technology,0.00


In [243]:
for i, r in sentiment_df.iterrows():
    if type(r['opening']) == str:
        print(i, r['opening'])
        sentiment_df.at[i, 'opening'] = 0
#        try:
#            sentiment_df.at[i, 'opening'] = locale.atoi(r['opening'].strip())
#        except:
#            print(i, r['opening'])
#            sentiment_df.at[i, 'opening'] = -1
#        print(i, type(r['opening']))
#        break

33031   


In [245]:
for i, r in sentiment_df.iterrows():
    if type(r['high']) == str:
        print(i, r['high'])
        sentiment_df.at[i, 'high'] = 0
#        try:
#            sentiment_df.at[i, 'closing'] = locale.atoi(r['closing'].strip())
#        except:
#            print(i, r['closing'])
#            sentiment_df.at[i, 'closing'] = -1
#        print(i, type(r['closing']))
#        break

33031   


In [248]:
for i, r in sentiment_df.iterrows():
    if type(r['closing_1']) == str:
        print(i, r['closing_1'])
        sentiment_df.at[i, 'closing_1'] = 0
#        try:
#            sentiment_df.at[i, 'closing'] = locale.atoi(r['closing'].strip())
#        except:
#            print(i, r['closing'])
#            sentiment_df.at[i, 'closing'] = -1
#        print(i, type(r['closing']))
#        break

33031   


In [250]:
sentiment_df.drop(['closing-_1'], inplace = True, axis=1)

In [253]:
def fix_price(s_df, col_name):
    for i, r in s_df.iterrows():
        if type(r[col_name]) == str:
            s_df.at[i, col_name] = 0
#            print(i, r[col_name])
#            try:
#                s_df.at[i, col_name] = locale.atoi(r['closing'].strip())
#            except:
#                print(i, r[col_name])
#                s_df.at[i, col_name] = -1

In [ ]:
fix_price(sentiment_df, 'high')

In [139]:
fix_price(sentiment_df, 'closing-2')

23612  601,000 
23613  355,000 
23614  11,750 
23615  271,200 
23616  20,900 
23617  34,900 
23618  43,350 
23619  8,470 
23620  51,500 
23621  36,250 
23622  78,300 
23623  14,650 
23624  116,000 
23625  190,500 
23626  81,200 
23627  27,000 
23628  5,530 
23629  383,000 
23630  4,770 
23631  29,700 
23632  164,000 
23633  4,880 
23634  4,880 
23635  179,000 
23636  78,300 
23637  17,250 
23638  270,000 
23639  251,000 
23640  168,000 
23641  39,500 
23642  270,000 
23643  116,000 
23644  913,000 
23645  72,200 
23646  43,150 
23647  7,890 
23648  31,400 
23649  8,570 
23650  52,800 
23651  37,100 
23652  14,200 
23653  39,050 
23654  155,500 
23655  61,200 
23656  19,000 
23657  27,650 
23658  33,750 
23659  12,850 
23660  78,300 
23661  116,000 
23662  18,600 
23663  24,350 
23664  2,315 
23665  4,985 
23666  17,200 
23667  53,500 
23668  16,400 
23669  62,200 
23670  25,800 
23671  7,920 
23672  30,500 
23673  17,300 
23674  21,450 
23675  63,100 
23676  28,500 
23677  27,400 
2367

In [254]:
korean_colnames = ['고가1',
 '고가2',
 '고가3',
 '고가4',
 '고가5',
 '고가6',
 '고가7',
 '고가8',
 '고가9',
 '고가10',
 '고가11',
 '고가12',
 '고가13',
 '고가14',
 '고가15',
 '고가16',
 '고가17',
 '고가18',
 '고가19',
 '고가20',
 '고가21',
 '고가22',
 '고가23',
 '고가24',
 '고가25',
 '고가26',
 '고가27',
 '고가28',
 '고가29',
 '고가30',
 '고가31',
 '고가32',
 '고가33',
 '고가34',
 '고가35',
 '고가36',
 '고가37',
 '고가38',
 '고가39',
 '고가40',
 '고가41',
 '고가42',
 '고가43',
 '고가44',
 '고가45',
 '고가46',
 '고가47',
 '고가48',
 '고가49',
 '고가50',]

In [255]:
for colnm in korean_colnames:
    fix_price(sentiment_df, colnm)

In [256]:
fix_price(sentiment_df, 'H25')
fix_price(sentiment_df, 'H50')

# Labeling

In [257]:
sentiment_df.query('closing_1 == " - "')

,author,source,ticker_symbol,company,report_date,report_date-1,report_date-2,report_title,opening,closing,...,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title,sentiment


In [258]:
## Labeling 1
sentiment_df.query('closing_1 <= closing and sentiment > 0')[['closing_1', 'closing']]

,closing_1,closing
49,2630,2670
50,15300,16000
51,8130,8490
53,8590,8920
62,8533,8700
...,...,...
39776,709000,756000
39783,10800,10950
39787,41800,42100
39795,181000,188000


In [259]:
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
sentiment_df['__label1'] = sentiment_df.apply(lambda x: 1 if x.closing_1 <= x.closing else 0, axis=1)

In [260]:
sentiment_df['__label2'] = sentiment_df.apply(lambda x: 1 if x.closing <= x.고가1 else 0, axis=1)

In [261]:
sentiment_df['__label3'] = sentiment_df.apply(lambda x: 1 if x.closing_1 <= x.고가1 else 0, axis=1)

In [262]:
sentiment_df.keys()

Index(['author', 'source', 'ticker_symbol', 'company', 'report_date',
       'report_date-1', 'report_date-2', 'report_title', 'opening', 'closing',
       'high', 'closing_1', 'closing_2', 'recommendation', '고가1', '고가2', '고가3',
       '고가4', '고가5', '고가6', '고가7', '고가8', '고가9', '고가10', '고가11', '고가12',
       '고가13', '고가14', '고가15', '고가16', '고가17', '고가18', '고가19', '고가20', '고가21',
       '고가22', '고가23', '고가24', '고가25', '고가26', '고가27', '고가28', '고가29', '고가30',
       '고가31', '고가32', '고가33', '고가34', '고가35', '고가36', '고가37', '고가38', '고가39',
       '고가40', '고가41', '고가42', '고가43', '고가44', '고가45', '고가46', '고가47', '고가48',
       '고가49', '고가50', 'H25', 'H50', 'en_report_title', 'sentiment',
       '__label1', '__label2', '__label3'],
      dtype='object')

In [263]:
sentiment_df.query('closing_1 <= closing and sentiment > 0')[['closing_1', 'closing', '고가1', '__label1', '__label2', '__label3']]

,closing_1,closing,고가1,__label1,__label2,__label3
49,2630,2670,2726.0,1,1,1
50,15300,16000,16050.0,1,1,1
51,8130,8490,8550.0,1,1,1
53,8590,8920,8950.0,1,1,1
62,8533,8700,8833.0,1,1,1
...,...,...,...,...,...,...
39776,709000,756000,757000.0,1,1,1
39783,10800,10950,10950.0,1,1,1
39787,41800,42100,42650.0,1,1,1
39795,181000,188000,188000.0,1,1,1


In [264]:
sentiment_df[['closing_1', 'closing', '고가1', '__label1', '__label2', '__label3']]

,closing_1,closing,고가1,__label1,__label2,__label3
0,51600,50000,51400.0,0,1,0
1,62600,62800,63200.0,1,1,1
2,77500,76000,77400.0,0,1,0
3,236000,232000,237500.0,0,1,1
4,317500,314000,321000.0,0,1,1
...,...,...,...,...,...,...
39795,181000,188000,188000.0,1,1,1
39796,10800,10950,10950.0,1,1,1
39797,124000,123500,124500.0,0,1,1
39798,8580,8770,8880.0,1,1,1


In [265]:
sentiment_df.query('__label1 == 1')

,author,source,ticker_symbol,company,report_date,report_date-1,report_date-2,report_title,opening,closing,...,고가48,고가49,고가50,H25,H50,en_report_title,sentiment,__label1,__label2,__label3
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,폴리실리콘 원가개선 필요한 시점,62700,62800,...,48200.0,46300.0,39150.0,65900,67300,Polysilicon cost improvement required,0.000000,1,1,1
6,박은경,삼성,A008770,호텔신라,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,마케팅을 적게 해도 수요는 작아지지 않는다,92100,94000,...,85000.0,81800.0,76500.0,111000,111000,"Even if you have less marketing, demand is not...",-0.020833,1,1,1
14,장문준,KB,A009240,한샘,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,무난한 4분기 실적 vs. 아직은 불확실한 업황,62500,66700,...,64400.0,63000.0,57900.0,78800,79300,4Q earnings vs.Still uncertainty,0.000000,1,1,1
16,김양재,KTB,A000660,SK하이닉스,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4Q19 Preview: 컨센서스 소폭 상회 전망,96000,94700,...,88500.0,84600.0,84700.0,103500,106000,4Q19 Preview: Consensus Sight,0.000000,1,1,1
17,김양재,KTB,A034220,LG디스플레이,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4Q19 Preview: 4Q19 저점 실적 회복 전망,16450,16350,...,13300.0,12900.0,11650.0,16800,16800,4q19 Preview: 4q19 Low-point performance recov...,0.000000,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39794,박혜진,대신,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,민영화터널의 끝이 보인다,10850,10950,...,0.0,0.0,0.0,0,0,The end of the privatization tunnel,0.000000,1,1,1
39795,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,182500,188000,...,0.0,0.0,0.0,0,0,Room and comfortable 2022,0.400000,1,1,1
39796,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,10850,10950,...,0.0,0.0,0.0,0,0,Welcome to the confusion of the confidence,0.800000,1,1,1
39798,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지",8840,8770,...,0.0,0.0,0.0,0,0,"Hidden pearls, 3Q Human & Renewable Energy",-0.083333,1,1,1


In [266]:
for k, r in sentiment_df.iterrows():
    if type(r['report_date']) == str:
#        print(k, r['report_date'], type(r['report_date']))
        sentiment_df.at[k, 'report_date'] = datetime.strptime(r['report_date'], '%m/%d/%y')
#        break

In [267]:
tmp = datetime.strptime('1/4/21', '%m/%d/%y')
type(tmp)

datetime.datetime

In [268]:
sentiment_df

,author,source,ticker_symbol,company,report_date,report_date-1,report_date-2,report_title,opening,closing,...,고가48,고가49,고가50,H25,H50,en_report_title,sentiment,__label1,__label2,__label3
0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 실적은 기대수준을 소폭 하회할 전망,51300,50000,...,40450.0,40550.0,36400.0,51400,51400,4Q09 earnings are expected to meet expectations,-0.100000,0,1,0
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,폴리실리콘 원가개선 필요한 시점,62700,62800,...,48200.0,46300.0,39150.0,65900,67300,Polysilicon cost improvement required,0.000000,1,1,1
2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,"견조한 고무사업, 그러나 부진한 페놀유도체",77300,76000,...,63200.0,63700.0,56300.0,82100,82100,"A solid rubber business, but sluggish phenol oil",0.000000,0,1,0
3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,20년 연초에 꾸는 꿈,237000,232000,...,301500.0,291500.0,284500.0,322500,351000,Dreams of 20 years,0.000000,0,1,1
4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 ESS 비용반영을 긍정적으로 보자,321000,314000,...,377500.0,367000.0,343500.0,391500,422500,Let's see a positive reflection of ESS costs i...,0.227273,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,2021-09-10 00:00:00,9/9/21,9/8/21,호실적과 편안한 2022년,182500,188000,...,0.0,0.0,0.0,0,0,Room and comfortable 2022,0.400000,1,1,1
39796,최정욱,하나금융,A316140,우리금융지주,2021-09-10 00:00:00,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,10850,10950,...,0.0,0.0,0.0,0,0,Welcome to the confusion of the confidence,0.800000,1,1,1
39797,심은주,하나금융,A271560,오리온,2021-09-10 00:00:00,9/9/21,9/8/21,저가 매수 유효,124500,123500,...,0.0,0.0,0.0,0,0,Low cost volume valid,0.000000,0,1,1
39798,박강호,대신,A229640,LS전선아시아,2021-09-10 00:00:00,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지",8840,8770,...,0.0,0.0,0.0,0,0,"Hidden pearls, 3Q Human & Renewable Energy",-0.083333,1,1,1


In [269]:
sentiment_df.to_pickle(PROJ_HOME+'/data/labeled_sentiment.pkl')

In [270]:
sentiment_df.query('closing_1 > 0 and closing > 0').to_pickle(PROJ_HOME+'/data/label_1_sentiment.pkl')

In [271]:
sentiment_df.columns

Index(['author', 'source', 'ticker_symbol', 'company', 'report_date',
       'report_date-1', 'report_date-2', 'report_title', 'opening', 'closing',
       'high', 'closing_1', 'closing_2', 'recommendation', '고가1', '고가2', '고가3',
       '고가4', '고가5', '고가6', '고가7', '고가8', '고가9', '고가10', '고가11', '고가12',
       '고가13', '고가14', '고가15', '고가16', '고가17', '고가18', '고가19', '고가20', '고가21',
       '고가22', '고가23', '고가24', '고가25', '고가26', '고가27', '고가28', '고가29', '고가30',
       '고가31', '고가32', '고가33', '고가34', '고가35', '고가36', '고가37', '고가38', '고가39',
       '고가40', '고가41', '고가42', '고가43', '고가44', '고가45', '고가46', '고가47', '고가48',
       '고가49', '고가50', 'H25', 'H50', 'en_report_title', 'sentiment',
       '__label1', '__label2', '__label3'],
      dtype='object')

In [272]:
features_df = sentiment_df[['author', 'source', 'ticker_symbol', 'company', 'report_date', 'recommendation', 'sentiment', '__label1', '__label2', '__label3']]

In [273]:
features_df.to_pickle(PROJ_HOME+'/data/labeled_minimum_features.pkl')

In [274]:
features_df['ticker_symbol'].value_counts()

A005930    522
A035420    403
A017670    374
A066570    374
A000660    373
          ... 
A001130      1
A006090      1
A201490      1
A029460      1
A006490      1
Name: ticker_symbol, Length: 1245, dtype: int64

In [275]:
features_df['report_date'].value_counts()

2020-05-18    348
2020-08-18    302
2020-11-16    269
2021-05-17    264
2020-11-17    264
             ... 
2020-01-19      3
2020-07-25      2
2020-03-29      2
2020-05-01      1
2021-05-19      1
Name: report_date, Length: 428, dtype: int64

In [276]:
features_df['recommendation'].value_counts()

BUY           30522
NR             6317
HOLD           2333
STRONG_BUY       67
SELL             16
REDUCE           12
Name: recommendation, dtype: int64

In [277]:
features_df['__label1'].value_counts()

1    22444
0    16823
Name: __label1, dtype: int64

In [278]:
features_df['__label2'].value_counts()

1    39017
0      250
Name: __label2, dtype: int64

In [279]:
features_df['__label3'].value_counts()

1    35213
0     4054
Name: __label3, dtype: int64

In [280]:
features_df

,author,source,ticker_symbol,company,report_date,recommendation,sentiment,__label1,__label2,__label3
0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,BUY,-0.100000,0,1,0
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,HOLD,0.000000,1,1,1
2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,HOLD,0.000000,0,1,0
3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,BUY,0.000000,0,1,1
4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,BUY,0.227273,0,1,1
...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,2021-09-10 00:00:00,BUY,0.400000,1,1,1
39796,최정욱,하나금융,A316140,우리금융지주,2021-09-10 00:00:00,BUY,0.800000,1,1,1
39797,심은주,하나금융,A271560,오리온,2021-09-10 00:00:00,BUY,0.000000,0,1,1
39798,박강호,대신,A229640,LS전선아시아,2021-09-10 00:00:00,BUY,-0.083333,1,1,1


In [281]:
sentiment_df.query('closing_1 > 0 and closing > 0')[['author', 'source', 'ticker_symbol', 'company', 'report_date', 'recommendation', 'sentiment', '__label1', '__label2', '__label3']]

,author,source,ticker_symbol,company,report_date,recommendation,sentiment,__label1,__label2,__label3
0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,BUY,-0.100000,0,1,0
1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,HOLD,0.000000,1,1,1
2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,HOLD,0.000000,0,1,0
3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,BUY,0.000000,0,1,1
4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,BUY,0.227273,0,1,1
...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,2021-09-10 00:00:00,BUY,0.400000,1,1,1
39796,최정욱,하나금융,A316140,우리금융지주,2021-09-10 00:00:00,BUY,0.800000,1,1,1
39797,심은주,하나금융,A271560,오리온,2021-09-10 00:00:00,BUY,0.000000,0,1,1
39798,박강호,대신,A229640,LS전선아시아,2021-09-10 00:00:00,BUY,-0.083333,1,1,1


In [282]:
sentiment_df[['closing_1', 'closing']]

,closing_1,closing
0,51600,50000
1,62600,62800
2,77500,76000
3,236000,232000
4,317500,314000
...,...,...
39795,181000,188000
39796,10800,10950
39797,124000,123500
39798,8580,8770
